In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.1.0


## Load data

In [4]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [5]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [6]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [7]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [8]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [9]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [10]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, random_state=42)

In [12]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [13]:
X_train = np.transpose(X_train, (0, 2, 1))
X_valid = np.transpose(X_valid, (0, 2, 1))
har_X_test = np.transpose(har_X_test, (0, 2, 1))

In [14]:
X_train.shape, y_train.shape

((5514, 9, 128), (5514,))

# Build RNN model

## Hyperparameters

In [29]:
batch_size = 200
n_epochs = 100
learning_rate = 0.003

seq_len = 128
n_channels = 9
n_neurons = 100
n_layers = 3

# outputs
n_classes = 6

In [30]:
n_batches = X_train.shape[0] // batch_size

## first RNN model

In [318]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [319]:
with tf.name_scope("rnn"):
    
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

In [320]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [321]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [322]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [323]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [324]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #shuffled_idx = np.random.permutation(X_train.shape[0])
        #X_random = X_train[shuffled_idx]
        #y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_first_rnn_model.ckpt")

Epoch:  0 Loss: 0.515187 Valid accuracy:  0.776387
Epoch:  10 Loss: 0.193966 Valid accuracy:  0.931447
Epoch:  20 Loss: 0.147235 Valid accuracy:  0.939608
Epoch:  30 Loss: 0.17161 Valid accuracy:  0.945049
Epoch:  40 Loss: 0.141158 Valid accuracy:  0.95049
Epoch:  50 Loss: 0.141284 Valid accuracy:  0.962459
Epoch:  60 Loss: 0.261729 Valid accuracy:  0.936344
Epoch:  70 Loss: 0.175188 Valid accuracy:  0.95593
Epoch:  80 Loss: 0.333655 Valid accuracy:  0.917301
Epoch:  90 Loss: 0.18773 Valid accuracy:  0.963003


In [325]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_first_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_first_rnn_model.ckpt
valid Accuracy:  0.962459
Test Accuracy:  0.910417


## RNN model with dropout

In [326]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [327]:
with tf.name_scope("rnn"):
    
    # Convolutional layers
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    drop_basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell, output_keep_prob=0.8)
    outputs, states = tf.nn.dynamic_rnn(drop_basic_cell, X, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

In [328]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [329]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [330]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [331]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [332]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #shuffled_idx = np.random.permutation(X_train.shape[0])
        #X_random = X_train[shuffled_idx]
        #y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_rnn_dropout_model.ckpt")

Epoch:  0 Loss: 0.505289 Valid accuracy:  0.776931
Epoch:  10 Loss: 0.304848 Valid accuracy:  0.904244
Epoch:  20 Loss: 0.154546 Valid accuracy:  0.941785
Epoch:  30 Loss: 0.155834 Valid accuracy:  0.945049
Epoch:  40 Loss: 0.16856 Valid accuracy:  0.95049
Epoch:  50 Loss: 0.16946 Valid accuracy:  0.948857
Epoch:  60 Loss: 0.216319 Valid accuracy:  0.943417
Epoch:  70 Loss: 0.273759 Valid accuracy:  0.929271
Epoch:  80 Loss: 0.179917 Valid accuracy:  0.955386
Epoch:  90 Loss: 0.251273 Valid accuracy:  0.95049


In [333]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_dropout_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_dropout_model.ckpt
valid Accuracy:  0.953754
Test Accuracy:  0.896844


## Bi-directional Basic RNN

In [334]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [335]:
with tf.name_scope("rnn"):
    
    fw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    
    bw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
        
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [336]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [337]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [338]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [339]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [340]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [341]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #shuffled_idx = np.random.permutation(X_train.shape[0])
        #X_random = X_train[shuffled_idx]
        #y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
             
        if epoch % 10 == 0:
            
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_rnn_model.ckpt")

Epoch:  0 Loss: 0.403904 Valid accuracy:  0.841132
Epoch:  10 Loss: 0.158717 Valid accuracy:  0.945049
Epoch:  20 Loss: 0.140548 Valid accuracy:  0.948313
Epoch:  30 Loss: 0.12571 Valid accuracy:  0.965724
Epoch:  40 Loss: 0.147911 Valid accuracy:  0.966812
Epoch:  50 Loss: 0.106514 Valid accuracy:  0.973341
Epoch:  60 Loss: 0.105223 Valid accuracy:  0.971708
Epoch:  70 Loss: 0.157873 Valid accuracy:  0.97062
Epoch:  80 Loss: 0.169719 Valid accuracy:  0.970076
Epoch:  90 Loss: 0.176848 Valid accuracy:  0.97062


In [342]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_model.ckpt
valid Accuracy:  0.972252
Test Accuracy:  0.899898


# Bi-directional Basic RNN with dropout

In [61]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [62]:
with tf.name_scope("rnn"):
    

    fw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    bw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [63]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [64]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [65]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [66]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [67]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [68]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #shuffled_idx = np.random.permutation(X_train.shape[0])
        #X_random = X_train[shuffled_idx]
        #y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
             
        if epoch % 10 == 0:
            
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_rnn_model.ckpt")

Epoch:  0 Loss: 0.591961 Valid accuracy:  0.724157
Epoch:  10 Loss: 0.160837 Valid accuracy:  0.93852
Epoch:  20 Loss: 0.132651 Valid accuracy:  0.954298
Epoch:  30 Loss: 0.19792 Valid accuracy:  0.945593
Epoch:  40 Loss: 0.153043 Valid accuracy:  0.952666
Epoch:  50 Loss: 0.229979 Valid accuracy:  0.944505
Epoch:  60 Loss: 0.188912 Valid accuracy:  0.957562
Epoch:  70 Loss: 0.227729 Valid accuracy:  0.958651
Epoch:  80 Loss: 0.264204 Valid accuracy:  0.945049
Epoch:  90 Loss: 0.179955 Valid accuracy:  0.952666


In [69]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_model.ckpt
valid Accuracy:  0.953754
Test Accuracy:  0.902613


# Bi-directional Basic RNN with dropout and early stopping

In [51]:
n_epoches=600

In [52]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [53]:
with tf.name_scope("rnn"):
    

    fw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
                   for _ in range(n_layers)]
    bw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
                   for _ in range(n_layers)]
    
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [54]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [55]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [56]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [57]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [58]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [59]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_rnn_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.374685 Valid accuracy:  0.852013
Epoch:  5 Loss:  0.17536 Valid accuracy:  0.9358
Epoch:  10 Loss:  0.158827 Valid accuracy:  0.942873
Epoch:  15 Loss:  0.122423 Valid accuracy:  0.95321
Epoch:  20 Loss:  0.116767 Valid accuracy:  0.960827
Epoch:  25 Loss:  0.112634 Valid accuracy:  0.955386
Epoch:  30 Loss:  0.127886 Valid accuracy:  0.964091
Epoch:  35 Loss:  0.118389 Valid accuracy:  0.965179
Epoch:  40 Loss:  0.129507 Valid accuracy:  0.97062
Epoch:  45 Loss:  0.171975 Valid accuracy:  0.960827
Epoch:  50 Loss:  0.148801 Valid accuracy:  0.964091
Epoch:  55 Loss:  0.16407 Valid accuracy:  0.966268
Epoch:  60 Loss:  0.182969 Valid accuracy:  0.969532
Epoch:  65 Loss:  0.270565 Valid accuracy:  0.949401
Epoch:  70 Loss:  0.207952 Valid accuracy:  0.957563
Epoch:  75 Loss:  0.231561 Valid accuracy:  0.961915
Epoch:  80 Loss:  0.182839 Valid accuracy:  0.965724
Epoch:  85 Loss:  0.203918 Valid accuracy:  0.966812
Epoch:  90 Loss:  0.21256 Valid accuracy:  0.968444
Ep

In [60]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_early_stopping_model.ckpt
valid Accuracy:  0.955386
Test Accuracy:  0.87377


# Three Basic RNN layers with  early stopping

In [31]:
# maximum number of epoch
n_epoches=600

In [42]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [43]:
with tf.name_scope("rnn"):
    

    cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
                 for _ in range(n_layers)]
    
    multi_cells = tf.contrib.rnn.MultiRNNCell(cells)
   
    outputs, states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [44]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [45]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [46]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [47]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [48]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [49]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_basic_rnn_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.539692 Valid accuracy:  0.759521
Epoch:  5 Loss:  0.247786 Valid accuracy:  0.920022
Epoch:  10 Loss:  0.160133 Valid accuracy:  0.94124
Epoch:  15 Loss:  0.156728 Valid accuracy:  0.958107
Epoch:  20 Loss:  0.205877 Valid accuracy:  0.945049
Epoch:  25 Loss:  0.174716 Valid accuracy:  0.958651
Epoch:  30 Loss:  0.18928 Valid accuracy:  0.951578
Epoch:  35 Loss:  0.160875 Valid accuracy:  0.960283
Epoch:  40 Loss:  0.141313 Valid accuracy:  0.967356
Epoch:  45 Loss:  0.179659 Valid accuracy:  0.965724
Epoch:  50 Loss:  0.190356 Valid accuracy:  0.967356
Epoch:  55 Loss:  0.177741 Valid accuracy:  0.952122
Epoch:  60 Loss:  0.182755 Valid accuracy:  0.965179
Epoch:  65 Loss:  0.17803 Valid accuracy:  0.973885
Epoch:  70 Loss:  0.206843 Valid accuracy:  0.972796
Epoch:  75 Loss:  0.215313 Valid accuracy:  0.972252
Epoch:  80 Loss:  0.221466 Valid accuracy:  0.972252
Epoch:  85 Loss:  0.226512 Valid accuracy:  0.972796
Epoch:  90 Loss:  0.230929 Valid accuracy:  0.97279

In [50]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_basic_rnn_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_basic_rnn_early_stopping_model.ckpt
valid Accuracy:  0.967356
Test Accuracy:  0.92433
